In [76]:
using CSV
using DataFrames
using Pkg
using Plots
using Dates
using MAT
using Statistics
using Printf

In [77]:
fe3 = BigFloat(642121496772645.10);
fe2 = BigFloat(688358979309308.22);
R_Lange = BigFloat(0.932829404530965376);


In [78]:
function arbTimetoEpoch(t,t0) #takes t, time elapsed in s since t0 (a date time) and returns epoch time corresponding to t
    return t + datetime2unix(t0) + 6*3600
end;

function readMoku(fileName)
    df = CSV.read(fileName, DataFrame; header = 10)
    ts = Array(df[:, 1]);  # % Time (s)
    f871s = Array(df[:, 3]); # Input 1 Frequency (Hz)
    f933s = Array(df[:, 8]);# Input 2 Frequency (Hz)
    regex = r"(\d{8}_\d{6})"
    dtStr = match(regex, fileName)
    t0 = DateTime(dtStr[1], "yyyymmdd_HHMMSS")
    for ii in 1:length(ts)
        ts[ii] = arbTimetoEpoch(ts[ii],t0)
    end
    return ts, f871s, f933s
end;

In [79]:
file1 =  raw"C:\Users\sanner_lab2\Documents\Data\Clock Runs\MokuPhasemeterData_20250402_131201.csv"
ts, f871s, f933s = readMoku(file1);

In [80]:
function nowToEpoch(time)
    return datetime2unix(DateTime(0,1,1))+time*(24*60*60) - 60*60*18
end;

function importMatlab(filename)
    matfile = matopen(filename)
    n = Int(read(matfile, "currentdatapointnr"))
    aomfrequsmatrix = read(matfile, "aomfrequsmatrix")
    taoms = aomfrequsmatrix[1:n-1, 1]  # Adjust for 1-based indexing in Julia
    
    # Convert times to epoch
    for ii in 1:length(taoms)
        taoms[ii] = nowToEpoch(taoms[ii])  # Assuming nowToEpoch is defined elsewhere
    end

    fe2aom = aomfrequsmatrix[1:n-1, 2]
    fe3aom = aomfrequsmatrix[1:n-1, 5]
    close(matfile)
    return taoms, fe2aom, fe3aom
end;

In [81]:
file2 = raw"C:\Users\sanner_lab2\Documents\Data\Clock Runs\aomfrequs-20250402_1.mat"
taoms, fe2aom, fe3aom  = importMatlab(file2);

In [82]:
#--------------------define length of interrogation-------------
tInt = 2.01
#-----------------------------------------------------------

# Assuming you already have the arrays: ts, f871s, f933s, taoms, fe2aom, fe3aom
# Also assuming they are vectors (1D arrays)
ni=0;
nf=0;

if ts[1] > taoms[1]  # Julia uses 1-based indexing
    println("case 1")
    for ii in 1:length(taoms)
        if (taoms[ii] - tInt)>ts[1]
            ni = ii
            # println(ni)
            break
        end
    end
    taoms = taoms[ni:end]
    fe2aom = fe2aom[ni:end]
    fe3aom = fe3aom[ni:end]
    println(ts[1] - taoms[1])
else
    println("case 2")
    for ii in 1:length(ts)
        if ts[ii] > taoms[1]
            ni = ii
            println(ni)
            break
        end
    end
    ts = ts[ni:end]
    f871s = f871s[ni:end]
    f933s = f933s[ni:end]
    println(ts[1] - taoms[1])
end

if ts[end] > taoms[end]
    println("case 1")
    for ii in 1:length(ts)
        if ts[ii] > taoms[end]
            nf = ii
            break
        end
    end
    ts = ts[1:nf-1]
    f871s = f871s[1:nf-1]
    f933s = f933s[1:nf-1]
    println(ts[end] - taoms[end])
else
    println("case 2")
    for ii in 1:length(taoms)
        if taoms[ii] > ts[end]
            nf = ii
            break
        end
    end
    taoms = taoms[1:nf-1]
    fe2aom = fe2aom[1:nf-1]
    fe3aom = fe3aom[1:nf-1]
    println(ts[end] - taoms[end])
end

case 1
-2.04400634765625
case 2
1.724611520767212


In [83]:
A = Array([1,2,3,4,5,6,7])
A[1:3]


3-element Vector{Int64}:
 1
 2
 3

In [84]:
n = length(taoms)
f871adj = zeros(n)
f933adj = zeros(n)

ii = 1
n1 = 1
n2 = 1

while ii <= n
    while ts[n1] < (taoms[ii] - tInt)
        n1 += 1
    end
    while ts[n2] < taoms[ii]
        n2 += 1
    end
    f871adj[ii] = mean(f871s[n1:(n2-2)])
    f933adj[ii] = mean(f933s[n1:(n2-2)])
    println((n2-2)-n1)
    ii += 1
end

57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
57
58
58
58
58
57
57
57
57
57
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
58
57
58
57
57
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
58
58
58
58
57
58
58
58
58
58
58
57
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
58
58
57
58
57
57
57
58
57
57
58
58
58
57
57
57
58
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
57
58
57
5

58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
58
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
58
57
58
58
57
58
58
58
58
58
58
58
57
58
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
57
58
58
57
57
57
58
57
58
58
58
57
58
58
57
58
58
58
57
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
58
57
58
58
58
57
58
58
58
58
57
58
58
58
58
58
58
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
57
58
58
58
57
58
58
58
57
58
58
58
58
58
57
58
57
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
58
5

In [85]:
'''
n = length(ts)
fe2adj = zeros(n)
fe3adj = zeros(n)

jj = 1
ii = 1

while ii < length(taoms)
    while ts[jj] < taoms[ii+1]
        fe2adj[jj] = ((fe2aom[ii+1] - fe2aom[ii]) / (taoms[ii+1] - taoms[ii])) * (ts[jj] - taoms[ii]) + fe2aom[ii]
        fe3adj[jj] = ((fe3aom[ii+1] - fe3aom[ii]) / (taoms[ii+1] - taoms[ii])) * (ts[jj] - taoms[ii]) + fe3aom[ii]
        jj += 1
        if jj > n
            break
        end
    end
    ii += 1
end
'''

'\'': ASCII/Unicode U+0027 (category Po: Punctuation, other)

In [86]:
### Must be plus or minus one depending on which side beat note is on
beatSign871 = +1
###-------------------------------------------
frep = 250000066.767
fceo = BigFloat(70e6)
@printf("%f", (fe2/2-fceo + beatSign871*mean(f871s)+80.0e6+(161.5e6)/2)/frep);
println()
n871 = round(Int,(fe2/2-fceo + beatSign871*mean(f871s)+80.0e6+(161.5e6)/2)/frep);
@printf("%d", n871)

1376718.000010
1376718

In [87]:

### Must be plus or minus one depending on which side beat note is on
beatSign933 = +1
###-------------------------------------------
frep = 250000066.767
@printf("%f", (fe3/2-fceo + beatSign933*mean(f933s)+80.0e6+100e6/2)/frep);
println()
n933 = round(Int,(fe3/2-fceo + beatSign933*mean(f933s)+80.0e6+100e6/2)/frep);
@printf("%d", n933)

1284243.000009
1284243

In [98]:
nuA = fceo .- beatSign871.*f871adj;
nuB = (n871/n933).*(fceo.-beatSign933.*f933adj);
nuC = nuA .- nuB;
ratio = nuC./(fe3/2 + mean(fe3aom)/2 + 80e6).+n871/n933;
mean(ratio) - (fe2+240e6+81.5e6)/(fe3+160e6+100e6)

-4.219064382916746375555857610352703373243096556452658284399193299755173213280176e-13

In [99]:
B = BigFloat(31.275/(1e-9*0.59930*9.2740100657e-24/6.626e-34)); # in mu T
e2Zee = 52.14*B^2*1e-3;
e3Zee = -2.082*B^2*1e-3;
e2BBR = -517e-18*fe2;
e3BBR = -73.6e-18*fe3;
e2Quad = BigFloat(0.049409);
e3Quad = BigFloat(-0.049409/92.1);

In [101]:
f933 = fe3/2 + mean(fe3aom)/2 + 80e6
e2Shift = e2Zee+e2BBR+e2Quad;
e3Shift = e3Zee+e3BBR+e3Quad;
R = (2*ratio.-(240e6/f933).-(fe2aom./f933).-(e2Shift/f933))./(2-160e6/f933.-fe3aom./f933.-e3Shift/f933);
println("---Ratio Residual-----")
println((1/mean(R) - R_Lange))
println(((1/mean(R) - R_Lange)/R_Lange*fe2), " Hz")
println("---------------------")

---Ratio Residual-----
1.400327440745548983180405443574986362630127241285057351291818895201272103870816e-16
0.103333788914500732173289821643572362556725465641346034773676831449895788328419 Hz
---------------------


In [ ]:
0.1033360763007933714452933378482080037273627805253253066584687119560738768660631 Hz
